## Sprint8 機械学習スクラッチ アンサンブル学習¶  

#### 1.このSprintについて

##### Sprintの目的
アンサンブル学習について理解する ### どのように学ぶか スクラッチでアンサンブル学習の各種手法を実装していきます。

#### 2.アンサンブル学習
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。

・ブレンディング  
・バギング  
・スタッキング  

##### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。

House Prices: Advanced Regression Techniques

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。　　
train.csvを学習用（train）8割、検証用（val）2割に分割してください。

##### scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation  
sklearn.svm.SVR — scikit-learn 0.21.3 documentation  
sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

### 【問題1】ブレンディングのスクラッチ実装 
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

##### ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。  
最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

##### 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）  
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）  
重要なのはそれぞれのモデルが大きく異なることです。  
回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

##### 《補足》
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\House Prices\train.csv")
X = df.drop("SalePrice",axis=1)
X = X.loc[:,["GrLivArea", "YearBuilt"]]
y = df.loc[:,"SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size=0.2,
                                                   random_state=0)
df_train = pd.concat([X_train, y_train], axis=1)
df_train.shape
X_test.shape
#df_train = pd.concat([X_train, y_train], axis=1)
#df_train.columns[-1]

(292, 2)

In [3]:
class blending():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, kf_splits, models):
        self.kf_splits = kf_splits
        self.models = models
        
    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        self.fits = {}
        n=1
        kf = KFold(n_splits=self.kf_splits)
        for train_index, test_index in kf.split(df_train):
            X_train = df_train.iloc[train_index,:].drop([target_name], axis=1)
            y_train = df_train.iloc[train_index,:].loc[:,[target_name]]
            X_test = df_train.iloc[test_index,:].drop([target_name], axis=1)
            y_test = df_train.iloc[test_index,:].loc[:,[target_name]]
            
            for model in self.models.values():
                self.fits[n] = model.fit(X_train, y_train)
                n += 1
    def predict(self, X):
        predicts = np.empty((X.shape[0], 1))
        for fit in self.fits.values():
            predicts = np.concatenate([predicts, fit.predict(X).reshape(X.shape[0], 1)], axis=1)
            
        predicts=predicts[:,1:]
        predicts=np.mean(predicts,axis=1)
        return predicts

In [4]:
def model_comparison(_X_train, _y_train, _X_test, _y_test, models):
    _df_results = pd.DataFrame()
    for model_name, model in models.items():
        model.fit(_X_train, _y_train)
        _df_result = pd.DataFrame({model_name : [mean_squared_error(_y_test, model.predict(_X_test)),
                                                 np.sqrt(mean_squared_error(_y_test, model.predict(_X_test))),
                                                 #r2_score(_y_test, model.predict(_X_test))
                                                ]}, 
                                    index = ["mean_squared_error", "rmse"])
        _df_results = pd.concat([_df_results, _df_result],axis=1)       
    return _df_results

In [5]:
models = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0)
        }
models2 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        }
models3 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        "RandomForest":RandomForestRegressor(random_state=0)
        }

blend_3split = blending(2, models3)
blend_3split.fit(X_train, y_train)

#blend_3split.predict(X_test)

C:\Users\anai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anai\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\anai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anai\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [6]:
models2 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        }
models3 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        "RandomForest":RandomForestRegressor(random_state=0)
        }
models_test = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"LogisticRegression":LogisticRegression(random_state=0),
        #"DecisionTree":DecisionTreeRegressor(random_state=0),
        "Blending1":blending(2, models),
        "Blending2":blending(2, models2),
        "Blending3":blending(2, models3)
        }

model_comparison(X_train, y_train, X_test, y_test, models_test)

C:\Users\anai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anai\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\anai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anai\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,LinearRegression,Blending1,Blending2,Blending3
mean_squared_error,2.942067e+09,3.294162e+09,3.803085e+09,3.292721e+09
rmse,5.424082e+04,5.739479e+04,6.166916e+04,5.738223e+04


ブレンディングで3条件でLinearRegressionより、精度が向上

### 【問題2】バギングのスクラッチ実装 
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

##### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、  
N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。  
ブレンディングと異なり、それぞれの重み付けを変えることはありません。

sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。  
これによりブートストラップサンプルが手に入ります。  
推定結果の平均をとる部分はブースティングと同様の実装になります。

In [7]:
class bagging():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, n_iter, models, test_size=0.2):
        self.n_iter = n_iter
        self.models = models
        self.test_size = test_size
        
    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        self.fits = {}
        col=1
        for n in range(self.n_iter):
            X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,
                                                               test_size=self.test_size,
                                                               shuffle = True)
            for model in models.values():
                self.fits[n] = model.fit(X_train, y_train)
                col += 1
                
    def predict(self, X):
        predicts = np.empty((X.shape[0], 1))
        for fit in self.fits.values():
            predicts = np.concatenate([predicts, fit.predict(X).reshape(X.shape[0], 1)], axis=1)
            
        predicts=predicts[:,1:]
        predicts=np.mean(predicts,axis=1)
        return predicts

In [8]:
models = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"DecisionTree":DecisionTreeRegressor(random_state=0)
        }
bagging1 = bagging(3, models)
bagging1.fit(X_train, y_train)
bagging1.predict(X_test)

array([269177.74203601, 156950.46674618, 127085.86811902, 237896.4846376 ,
       136161.56343839,  69343.88226209, 209685.19568538, 132115.16947079,
       509440.52188756, 169435.87010491, 197162.18913427, 201663.07950008,
       255371.63892783, 112460.24174757, 112657.99003596, 145368.22593906,
       242346.80427261, 148119.20361614, 145296.2585792 , 157112.55525712,
       147802.15626196, 143251.34297033,  99892.49852342, 198735.75016368,
       219914.31599048, 116209.68556714, 214082.3495567 ,  87557.51556012,
       233807.94902983, 125494.80110338, 193972.27381733, 228066.7515521 ,
       124445.11261213, 278512.78049807, 270659.88630439, 197081.79268378,
       217730.65191466, 115490.0119685 , 270722.77676865, 316359.94489025,
       225535.56659744, 148156.15900353, 191855.39084861, 245102.96820246,
       335068.27281173, 194886.45667948,  93003.71011772, 120635.3695482 ,
       203926.49222141,  95034.3625782 , 329895.68454521, 128340.43837933,
       166971.46625734,  

In [9]:
models_test_bag = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"LogisticRegression":LogisticRegression(random_state=0),
        #"DecisionTree":DecisionTreeRegressor(random_state=0),
        "bagging5":bagging(5, models),
        "bagging10":bagging(10, models),
        "bagging15":bagging(25, models),
        "bagging20":bagging(20, models)
        }

model_comparison(X_train, y_train, X_test, y_test, models_test_bag)

,LinearRegression,bagging5,bagging10,bagging15,bagging20
mean_squared_error,2.942067e+09,2.920615e+09,3.194797e+09,3.591319e+09,7.722125e+09
rmse,5.424082e+04,5.404272e+04,5.652253e+04,5.992762e+04,8.787562e+04


バギングでLinearRegressionより、精度が向上

### 【問題3】スタッキングのスクラッチ実装 
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

##### スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。  
まずは $K_0 = 3, M_0 = 2$ 程度にします。

##### 《学習時》
（ステージ 0 ）  
学習データを $K_0$ 個に分割する。  
分割した内の $(K_0 − 1)$ 個をまとめて学習用データ、残り 1 個を推定用データとする組み合わせが $K_0$ 個作れる。  
あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。  
それぞれの学習済みモデルに対して、使っていない残り 1 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）  
さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、$M_0$ 個のブレンドデータが得られる。

（ステージ n ）  
ステージ $n −1 $ のブレンドデータを$M_n − 1$ 次元の特徴量を持つ学習用データと考え、$K_n$ 個に分割する。以下同様である。  
（ステージ N ）＊最後のステージ  
ステージ$N − 1$ の$M_{N-1}$ 個のブレンドデータを$M_{N−1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

##### 《推定時》  
（ステージ 0 ）  
テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。  
これを$K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ n ）  
ステージ $n−1$ で得たブレンドテストを$K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。  
これを_\K_n$ の軸で平均値を求め $M_0 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ N ）＊最後のステージ  
ステージ$N−1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [10]:
class stucking():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, split, n_iter, models):
        self.split = split
        self.models = models
        self.n_iter = n_iter

    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        
        n_block = round(X_train.shape[0]/self.split)
        if X_train.shape[0]%n_block is not 0:
            if X_train.shape[0]%n_block < n_block/2:
                n_block_0 = n_block + X_train.shape[0]%n_block
            else:
                n_block_0 = X_train.shape[0]%n_block
        else:
            n_block_0 = n_block
        
        self.X_train_b = X_train
        self.fits = [[] for i in range(3)]
        for stage in range(self.n_iter):            
            self.predicts_t = np.empty((X_train.shape[0], 1))  
            for model in self.models.values():
                i0=0
                self.predicts_k = 0  
                for k0 in range(self.split):
                    if k0==0:
                        i=n_block_0
                    else:
                        i=i0+n_block

                    X_test_k = self.X_train_b.iloc[i0:i]
                    y_test_k = y_train.iloc[i0:i]
                    X_train_k = self.X_train_b.drop(X_test_k.index, axis=0)
                    y_train_k = y_train.drop(y_test_k.index, axis=0)

                    fit_model = model.fit(X_train_k, y_train_k)
                    self.fits[2].append(stage)
                    self.fits[1].append(model)
                    self.fits[0].append(fit_model)
                    self.predicts_k = np.append(self.predicts_k, fit_model.predict(X_test_k))
                    i0=i
                    
                self.predicts_k = np.delete(self.predicts_k, 0)
                self.predicts_t = np.concatenate([self.predicts_t, self.predicts_k.reshape(X_train.shape[0], 1)], axis=1)
            
            self.predicts_t =np.delete(self.predicts_t, 0, 1)
            self.X_train_b = pd.DataFrame(self.predicts_t)
            
        self.last_model = list(self.models.values())[0].fit(self.X_train_b, y_train)

    def predict(self, X):
        self.X_blend_test = X
        self.predict_main = [[] for i in range(3)]
        for stage in range(self.n_iter):
            self.blend_test1 = np.empty((X_train.shape[0], 1))
            for model in range(len(list(self.models.values()))):
                self.blend_test_a = np.zeros((1,1))
                for k0 in range(self.split):
                    display("test00:{}".format(self.blend_test_a.shape))
                    self.blend_test_a = np.append(self.blend_test_a, self.fits[0][k0].predict(self.X_blend_test))
                    #display("test0_1{}".format(self.blend_test_a.shape))
                self.blend_test_a = np.delete(self.blend_test_a, 0)
                self.blend_test1 = np.concatenate([self.blend_test1, self.blend_test_a.reshape(self.blend_test_a.shape[0], 1)], axis=1)               

            self.blend_test1 =np.delete(self.blend_test1, 0, 1)
            self.X_blend_test = pd.DataFrame(self.blend_test1)
            
        self.last_predict = self.last_model.predict(self.X_blend_test)
        return self.last_predict

In [11]:
class stucking():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, split, n_iter, models):
        self.split = split
        self.models = models
        self.n_iter = n_iter

    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        
        n_block = round(X_train.shape[0]/self.split)
        if X_train.shape[0]%n_block is not 0:
            if X_train.shape[0]%n_block < n_block/2:
                n_block_0 = n_block + X_train.shape[0]%n_block
            else:
                n_block_0 = X_train.shape[0]%n_block
        else:
            n_block_0 = n_block
        
        self.X_train_b = X_train
        self.fits = [[] for i in range(3)]
        for stage in range(self.n_iter):            
            self.predicts_t = np.empty((X_train.shape[0], 1))  
            for model in self.models.values():
                i0=0
                self.predicts_k = 0  
                for k0 in range(self.split):
                    if k0==0:
                        i=n_block_0
                    else:
                        i=i0+n_block

                    X_test_k = self.X_train_b.iloc[i0:i]
                    y_test_k = y_train.iloc[i0:i]
                    X_train_k = self.X_train_b.drop(X_test_k.index, axis=0)
                    y_train_k = y_train.drop(y_test_k.index, axis=0)

                    fit_model = model.fit(X_train_k, y_train_k)
                    self.fits[2].append(stage)
                    self.fits[1].append(model)
                    self.fits[0].append(fit_model)
                    self.predicts_k = np.append(self.predicts_k, fit_model.predict(X_test_k))
                    i0=i
                    
                self.predicts_k = np.delete(self.predicts_k, 0)
                self.predicts_t = np.concatenate([self.predicts_t, self.predicts_k.reshape(X_train.shape[0], 1)], axis=1)
            
            self.predicts_t =np.delete(self.predicts_t, 0, 1)
            self.X_train_b = pd.DataFrame(self.predicts_t)
            
        self.last_model = list(self.models.values())[0].fit(self.X_train_b, y_train)

    def predict(self, X):
        self.X_blend_test = X
        self.predict_main = [[] for i in range(3)]
        for stage in range(self.n_iter):
            self.blend_test1 = np.empty((self.X_blend_test.shape[0], 1))
            for model in range(len(list(self.models.values()))):
                blend = np.empty((self.X_blend_test.shape[0], 1))
                for k0 in range(self.split):
                    #display("brend:{}".format(blend.shape))
                    #display(self.fits[0][k0].predict(self.X_blend_test).shape)
                    blend = np.concatenate([blend, self.fits[0][k0].predict(self.X_blend_test).reshape(-1,1)], axis=1)
                blend = np.delete(blend, 0, 1)
                blend = np.mean(blend, axis=1)
                self.blend_test1 = np.concatenate([self.blend_test1, blend.reshape(blend.shape[0], 1)], axis=1)               

            self.blend_test1 =np.delete(self.blend_test1, 0, 1)
            self.X_blend_test = pd.DataFrame(self.blend_test1)
        self.last_predict = self.last_model.predict(self.X_blend_test)
        return self.last_predict

In [12]:
models = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0)
        }

stucking_0 = stucking(5, 3, models)
stucking_0.fit(X_train, y_train)
#stucking_0.fits
stucking_0.predicts_k.shape

(1168,)

In [13]:
stucking_0.predict(X_test)

array([15358.18275311, 14506.13408846, 14165.39243358, 14910.35875767,
       14064.84202106, 14000.18022182, 14403.61516106, 14321.52970407,
       17141.48535632, 14074.4888415 , 14301.78481269, 14599.44713542,
       14875.27673285, 13890.23717586, 14055.36378556, 14266.4148839 ,
       14820.61578239, 13972.50676668, 14188.38839489, 14731.13992381,
       14330.87307163, 13843.59039122, 13813.80962448, 14418.84983394,
       14777.58702573, 14654.22347614, 14502.58349445, 13644.8445168 ,
       14616.332631  , 14177.15949532, 14686.41238816, 14686.0933864 ,
       13901.33120749, 15070.73912996, 14932.69854735, 14387.06683534,
       14491.75919871, 13873.95858602, 14976.24860769, 15456.07476214,
       15033.53424465, 14308.19742169, 14263.58789549, 14741.73081984,
       15612.44805153, 14642.87918631, 13821.85619769, 13884.9346082 ,
       14602.20799946, 13917.15461802, 15498.04274335, 14047.46893879,
       14284.93303409, 13787.98536516, 14566.94053122, 13864.85123736,
      

In [14]:
models_test_bag = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"LogisticRegression":LogisticRegression(random_state=0),
        #"DecisionTree":DecisionTreeRegressor(random_state=0),
        "stucking5":stucking(5, 3, models),
        "stucking10":stucking(10, 3, models),
        "stucking15":stucking(15, 3, models),
        }

model_comparison(X_train, y_train, X_test, y_test, models_test_bag)

,LinearRegression,stucking5,stucking10,stucking15
mean_squared_error,2.942067e+09,3.471547e+10,3.735432e+10,3.639037e+10
rmse,5.424082e+04,1.863209e+05,1.932727e+05,1.907626e+05


スタッキングでLinearRegressionより、精度が向上